In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from common_import import *


def check_config_experiment(whitelist_checkpoint):
    user = getpass.getuser()
    motionbert_root = f'/home/{user}/codes/MotionBERT'
    for checkpoint in whitelist_checkpoint:
        experiment_path = f'{motionbert_root}/experiments/{checkpoint}.txt'
        config_path = f'{motionbert_root}/configs/pose3d/{checkpoint}.yaml'
        print(f'{checkpoint}')
        if os.path.exists(experiment_path): print(f'Experiment path: {experiment_path}')
        else:
            print(f'[No experiment path] {experiment_path}')
            with open(experiment_path, 'w') as f: pass
        if os.path.exists(config_path): print(f'Config# path: {config_path}')
        else: print(f'[No config path] {config_path}')

### Evaluate single experiment

In [35]:
checkpoint_name = 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_prev'
check_config_experiment([checkpoint_name])

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_prev
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_prev.txt
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_prev.yaml


In [38]:
# load model
args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
model_pos, chk_filename, checkpoint = load_model(opts, args)
# load dataset
subset = '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN'
subset_path = f'/home/{user}/codes/MotionBERT/data/motion3d/yaml_files/{subset}.yaml'
print(subset_path)
if 'PCL' in subset: args.fix_orientation_pred = True # load dataset
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_prev/best_epoch.bin
/home/hrai/codes/MotionBERT/data/motion3d/yaml_files/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN.yaml
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_revolute...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_revolute...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:28<00:00,  2.76it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 70.30790621490644 mm
Protocol #2 Error (P-MPJPE): 56.36720767305519 mm
----------------------------------------


In [39]:
# save results
save = True
if save:
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    save_path = os.path.join(save_folder, f'{subset}.pkl')
    results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
    savepkl(results_dict, save_path)

In [33]:
# update result_dict
update_result_dict = True
if update_result_dict:
    result_dict = readpkl('result_dict.pkl')
    try: del result_dict[checkpoint_name][subset]
    except: print('subset not found')
    savepkl(result_dict, 'result_dict.pkl')

### Evaluate and save all experiments

In [ ]:
result_list = os.listdir('saved_results')
checkpoint_list = os.listdir('checkpoint/pose3d')
whitelist_checkpoint = []
for item in os.listdir('configs/pose3d'):
    checkpoint = item.split('.yaml')[0]
    if 'cam_no_factor' not in checkpoint: continue
    if checkpoint in blacklist_checkpoint: continue
    if checkpoint not in checkpoint_list:
        print(f'[Need to be trained] {checkpoint}')
        continue
    if checkpoint not in experiment_list:
        whitelist_checkpoint.append(checkpoint)
        # create empty file
        with open(f'experiments/{checkpoint}.txt', 'w') as f:
            pass
    else:
        subset_list = []
        with open(f'experiments/{checkpoint}.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                subset = line.split('\n')[0]
                subset_list.append(subset)
        if len(subset_list) == 0:
            print(f'[No subset fou  d] {checkpoint}')
            whitelist_checkpoint.append(checkpoint)
        for subset in subset_list:
            if not os.path.exists(f'saved_results/{checkpoint}/{subset}.pkl'):
                print(f'[Result not found] {checkpoint} - {subset}')
                if checkpoint not in whitelist_checkpoint:
                    whitelist_checkpoint.append(checkpoint)

print('\nCheckpoints to evaluate:')
whitelist_checkpoint

In [ ]:
whitelist_checkpoint = [
    #'MB_train_h36m_gt_cam_no_factor',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering_fix_orientation_pred_tr_s1_ts_s5678',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering_fix_orientation_pred',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678',
]

# check experiment file
check_config_experiment(whitelist_checkpoint)

In [ ]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in tqdm(glob(experiment_root+'/*.txt')):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if not os.path.exists(save_path): continue
        results_dict = readpkl(save_path)
        #print(checkpoint_name, subset, results_dict.keys())
        if 'gts_all' in results_dict.keys(): del results_dict['gts_all']
        if 'inputs_all' in results_dict.keys(): del results_dict['inputs_all']
        savepkl(results_dict, save_path)

  0%|          | 0/97 [00:00<?, ?it/s]

 31%|███       | 30/97 [10:41<22:45, 20.39s/it]

In [5]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in glob(experiment_root+'/*.txt'):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    if checkpoint_name in blacklist_checkpoint: continue
    if len(whitelist_checkpoint) > 0 and checkpoint_name not in whitelist_checkpoint: continue
    if 'cam_no_factor' not in checkpoint_name: continue
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)

    # load model
    args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
    model_pos, chk_filename, checkpoint = load_model(opts, args)

    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        if '3DHP' not in subset: continue
        if 'PCL' in subset: args.fix_orientation_pred = True
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if os.path.exists(save_path):
            print(f'{save_path} exists')
            continue
        try:
            # load dataset
            args.subset_list = [subset]
            train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
            # evaluation
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            # save results
            results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
            savepkl(results_dict, save_path)
        except Exception as e:
            print(f'Error: {e}')


MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678


Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678/best_epoch.bin
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL-TEST_TS1_6
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_pcl...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_pcl...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 6/6 [00:04<00:00,  1.26it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 98.1236313420414 mm
Protocol #2 Error (P-MPJPE): 66.67291445925403 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL-TEST_TS1_6_UNIV
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_pcl...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_pcl...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 6/6 [00:02<00:00,  2.29it/s]


...

No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): nan mm
Protocol #2 Error (P-MPJPE): nan mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_TS1_6_UNIV
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_revolute_no_Rz...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_revolute_no_Rz...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 6/6 [00:02<00:00,  2.24it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): nan mm
Protocol #2 Error (P-MPJPE): nan mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE_NO_RZ-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_revolute_no_Rz...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_revolute_no_Rz...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:22<00:00,  2.97it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): nan mm
Protocol #2 Error (P-MPJPE): nan mm
----------------------------------------


### Cherrypick e1, e1 -> result_dict.pkl

In [10]:
os.chdir('/home/hrai/codes/MotionBERT')

In [11]:
result_dict = readpkl('result_dict.pkl')
for item in whitelist_checkpoint:
    del result_dict[item]
savepkl(result_dict, 'result_dict.pkl')

KeyError: 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original'

In [40]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']

def update_result_dict(blacklist_checkpoint):
    if os.path.exists('result_dict.pkl'):
        print('Loading result_dict.pkl')
        result_dict = readpkl('result_dict.pkl')
    else:
        result_dict = {}
    result_root = '/home/hrai/codes/MotionBERT/saved_results'
    checkpoint_list = os.listdir(result_root)
    for checkpoint in tqdm(checkpoint_list):
        if checkpoint in blacklist_checkpoint: continue # skip
        if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
        checkpoint_root = os.path.join(result_root, checkpoint)
        subset_list = glob(checkpoint_root+'/*.pkl')
        for item in subset_list:
            subset = os.path.basename(item).split('.pkl')[0]
            if subset in result_dict[checkpoint].keys():
                #print(f'{subset} exists')
                continue
            result = readpkl(item)
            result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}
        # remove key if not in subset_list
        for key in list(result_dict[checkpoint].keys()):
            if key not in [os.path.basename(item).split('.pkl')[0] for item in subset_list]:
                del result_dict[checkpoint][key]

    savepkl(result_dict, 'result_dict.pkl')

update_result_dict(blacklist_checkpoint)

Loading result_dict.pkl


  0%|          | 0/85 [00:00<?, ?it/s]

100%|██████████| 85/85 [00:00<00:00, 186.77it/s]


In [ ]:
import prettytable

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Checkpoint', 'Subset', 'E1', 'E2']

# 데이터를 리스트에 수집
data = []
for key in result_dict.keys():
    for subset in result_dict[key].keys():
        if 'UNIV' not in subset: 
            continue
        e1 = float(result_dict[key][subset]['e1'])
        e2 = float(result_dict[key][subset]['e2'])
        data.append([key, subset, e1, e2])

# 데이터를 Subset 1순위, E1 2순위로 정렬
data_sorted = sorted(data, key=lambda x: (x[1], x[2]))

# 정렬된 데이터를 테이블에 추가
for row in data_sorted:
    pt.add_row([row[0], row[1], f'{row[2]:.2f}', f'{row[3]:.2f}'])

# 테이블 출력
print(pt)
